In [ ]:
#!pip install boltons

In [ ]:
#!pip install transformers

In [ ]:

from functools import partial
from pathlib import Path
from textwrap import wrap

import nltk
import pandas as pd
#from boltons.iterutils import windowed
from tqdm import tqdm, tqdm_notebook

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Model, AdamW, WarmupLinearSchedule
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler
import random
from IPython.core.display import display, HTML

#from google_drive_downloader import GoogleDriveDownloader as gdd

tqdm.pandas()
nltk.download('punkt', quiet=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


In [ ]:

def sample_text(model, seed='Weight loss can be achieved by', n_words=500):
    """Generate text from a trained model."""
    model.eval()
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
    
    text = tokenizer.encode(seed)
    inputs, past = torch.tensor([text]), None

    with torch.no_grad():
        for _ in tqdm_notebook(range(n_words), leave=False):
            logits, past = model(inputs.to(device), past=past)

            log_probs = F.softmax(logits[:, -1], dim=-1)
            inputs = torch.multinomial(log_probs, 1)
            
            text.append(inputs.item())

    return tokenizer.decode(text)


def pretty_print(text):
    """Wrap text for nice printing."""
    to_print = ''
    for paragraph in text.split('\n'):
        to_print += '\n'.join(wrap(paragraph))
        to_print += '\n'
    print(to_print)

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [ ]:
seed = 'Weight loss can be achieved by'  #@param {type:"string"}
n_words = 100  #@param {type:"integer"}
text = sample_text(model, seed=seed, n_words=n_words)
pretty_print(text)

In [ ]:
df = pd.read_csv('merged.csv')[['title', 'text', 'label']]


In [ ]:
df

In [ ]:
df_sample = df.sample(100)

In [ ]:
df_sample

In [ ]:
rows = []
generation = []
n_words = 200
for i in range(len(df_sample)):
    title = df_sample.title.iloc[i]
    original = df_sample.text.iloc[i]
    new = sample_text(model, seed=title, n_words=n_words)
    label = df_sample.label.iloc[i]
    generation.append(new)
    
    row = f"""
            <tr>
            <td>{title}&nbsp;</td>
            <td>{original}&nbsp;</td>
            <td>{new}&nbsp;</td>
            </tr>
            """
    rows.append(row)
            
rows_joined = '\n'.join(rows)
table = f"""
<table>
<col width="150">
<col width="480">
<col width="480">
<tbody>
<tr>
<td><b>Title</b>&nbsp;</td>
<td><b>Original</b>&nbsp;</td>
<td><b>Generated</b>&nbsp;</td>
</tr>
{rows_joined}
</tbody>
</table>
"""
    
    



In [ ]:
display(HTML(table))    

In [ ]:
df_sample['generated'] = generation

In [ ]:
df_sample

# Using article titles to generate new text

### Title 

In [ ]:
i = random.randint(0,99)

In [ ]:
pretty_print(df_sample.title.iloc[i])

### Original Text

In [ ]:
pretty_print(df_sample.text.iloc[i])

### Generated Text

In [ ]:
pretty_print(df_sample.generated.iloc[i])